<a href="https://colab.research.google.com/github/mgoncerz/SeparationPowerTool/blob/master/automatedTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Test settings:

In [0]:
data_file = "data.csv"
csv_delimiter = ','

pearson_coeff_thresholds = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3]
cutoff_values = [2,3,4,5,6,7,8,9,10,15,20]

columns_to_skip = ['', 'has_MCParticle', 'is_downstream_reconstructible', 'has_MCParticle_not_electron', 'is_downstream_reconstructible_not_electron', 'is_true_seed', 'seed_mva_value']


#use datarow['variable name']
def general_selection(datarow):
  if datarow['has_MCParticle'] == 'True' and datarow['has_MCParticle_not_electron'] == 'True':
    return True
  else:
    return False

def signal_definition(datarow):
  
  if datarow['is_downstream_reconstructible'] == 'True':
    return True
  else:
    return False

Load data:

In [19]:
from csv import DictReader
import numpy

with open(data_file, "r") as datafile:

  datastream = DictReader(datafile)

  variable_names = [name for name in datastream.fieldnames if name not in columns_to_skip]
  signal_data = {name : [] for name in variable_names}
  background_data = {name : [] for name in variable_names}
  print(variable_names)
  #read from file and split into signal and background
  for datarow in datastream:

    if general_selection(datarow):

      if signal_definition(datarow):
        for variable in variable_names:
          signal_data[variable].append(float(datarow[variable]))

      else:
        for variable in variable_names:
          background_data[variable].append(float(datarow[variable]))

  #convert to numpy arrays and normalize
  for variable in variable_names:
    signal_data[variable] = numpy.array(signal_data[variable], dtype=float)
    background_data[variable] = numpy.array(background_data[variable], dtype=float)

    signal_data[variable] = (signal_data[variable] - signal_data[variable].mean()) / signal_data[variable].std()
    background_data[variable] = (background_data[variable] - background_data[variable].mean()) / background_data[variable].std()

['seed_chi2PerDoF', 'seed_p', 'seed_pt', 'seed_nLHCbIDs', 'seed_nbIT', 'seed_nLayers', 'seed_x', 'seed_y', 'seed_tx', 'seed_ty']


Plot normalized data: